## Load libraries

### Need arcpy license and geostatistical analyst license to compare against kriging

Current citation

Lamb, D. S. 2018. Random Forest Trees for Spatial Interpolation. Presented at the Meeting of the Association of American Geographers, New Orleans, LA.


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from scipy import spatial
import arcpy
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point, shape, mapping
from scipy.interpolate import Rbf
%matplotlib inline

## Functions for formatting map output if needed

In [ ]:
def meters_formatter(x, p):
    strRes = '{:,}m'.format(int(x)) 
    return strRes

def scaleBar(x,y,mapdistance,ax,trans,subdivision=1,height=.02):
    """x - lower left corner of arrow in trans coordinates
       y - lower left corner of arrow in trans coordiantes
       mapdistance - maximum distance to show on the scalebar
       ax - axes to add patch and text
       trans - transformation the coordinates are in
       subdivision - number of subdivisions to show in the scalebar
       height - height of the bar part of the scalebar"""
    xmin, xmax = ax.get_xlim() #returns left,right
    abs_width = abs(xmax-xmin)
    length = 1.0/abs_width * mapdistance
    if subdivision > 1.0:
        sublength = float(length)/subdivision
        fColor = 'black'
        subx = x
        for i in range(0,subdivision):
            ax.add_patch(mpatches.Rectangle((subx,y), sublength, height, transform=trans,facecolor=fColor,edgecolor='black',lw=.5,clip_on=False))
            subx += sublength
            if fColor == 'black':
                fColor = 'white'
            else:
                fColor = 'black'
            
    else:
        ax.add_patch(mpatches.Rectangle((x,y), length, height, transform=trans,facecolor='black',edgecolor='black',clip_on=False))
        
    ax.text(x,y+height*1.5,'0',transform=trans,ha='center')
    ax.text(x+length,y+height*1.5,meters_formatter(mapdistance,None),transform=trans,ha='center')
    
def northArrowPatch(x,y,width,height,ax,trans):
    """x - lower left corner of arrow in trans coordinates
       y - lower left corner of arrow in trans coordiantes
       width - estimated width of arrow in trans coordiantes
       height - estimated height of arrow in trans coordiantes
       ax - axes to add patch and text
       trans - transformation the coordinates are in"""
    verts = []
    codes = []
    for part in range(0,4):
        if part == 0:
            verts.append((x,y))
            codes.append(Path.MOVETO)
        if part == 1:
            verts.append((x+(width/2.0),y-(height)))
            codes.append(Path.LINETO)
        if part == 2:
            verts.append((x,y-(height-(1.0/5.0)*height)))
            codes.append(Path.LINETO)
        if part == 3:
            verts.append((x-(width/2.0),y-(height)))
            codes.append(Path.LINETO)
            verts.append((x,y))
            codes.append(Path.CLOSEPOLY)

    path = Path(verts, codes)
    northPatch = mpatches.PathPatch(path, facecolor='k', lw=0,transform=trans,clip_on=False)
    ax.add_patch(northPatch)
    lbl = ax.text(x,y+((1.0/5.0)*height),'N',transform=trans,ha='center')
    return northPatch,lbl

## Simulate surface gaussian random noise, including an explanatory variable.

In [ ]:
n,m = 100,100
F = 2
x_c = np.linspace(1,n,n)

y_c = np.linspace(1,m,m)
X_c, Y_c = np.meshgrid(x_c,y_c)

i = np.minimum(X_c-1,n-X_c+1)
j = np.minimum(Y_c-1,m-Y_c+1)

isq = np.square(i)
jsq = np.square(j)

H = np.exp(-.5 * (isq+jsq) / (F**2))
Z = np.real(np.fft.ifft2(H*np.fft.fft2(np.random.randn(n,m))))
ya = np.random.rand(m)*100
xa = np.random.rand(n)*100
adjust =np.sqrt(xa**2+ya**2)
rbf = Rbf(xa, ya, adjust, epsilon=5)
ZI = rbf(X_c, Y_c)
rows={"xcoord":[],"ycoord":[],"zval":[],"dv":[],"dv2":[],"ROW":[],"COL":[]}
for i in range(n):
    row = X_c[i]
    #print row
    for j in range(n):
        rows['xcoord'].append(row[j])
        rows['ycoord'].append(Y_c[i][j])
        rows['dv'].append(Z[i][j])
        rows['dv2'].append(ZI[i][j])
        rows['zval'].append(Z[i][j]*ZI[i][j])
        rows['ROW'].append(i)
        rows['COL'].append(j)
df = pd.DataFrame(rows)
gridz= df.pivot('ROW','COL','zval')
#print grid

ax = sns.heatmap(gridz,cmap='Greys',xticklabels=False, yticklabels=False)

ax.set_aspect('equal')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("Simulated Gaussian Noise")
plt.tight_layout()

## Plot out the trend surface / explanatory variable

In [ ]:
gridz= df.pivot('ROW','COL','dv2')
#print grid

ax = sns.heatmap(gridz,cmap='Greys',xticklabels=False, yticklabels=False)

ax.set_aspect('equal')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("Trend Variable")
plt.tight_layout()

## Plot out figure

In [ ]:
fig, ((ax1, ax2,ax3)) = plt.subplots(1,3)
fig.set_size_inches(8, 4)
gridz= df.pivot('ROW','COL','zval')
sns.heatmap(gridz,cmap='Greys',xticklabels=False, yticklabels=False,ax=ax1,cbar=False)
ax1.set_aspect('equal')
ax1.set_xlabel("")
ax1.set_ylabel("")
ax1.set_title("Dependent Variable")
gridv= df.pivot('ROW','COL','dv')
sns.heatmap(gridv,cmap='Greys',xticklabels=False, yticklabels=False,ax=ax2,cbar=False)
ax2.set_aspect('equal')
ax2.set_xlabel("")
ax2.set_ylabel("")
ax2.set_title("Independent Variable")
gridt= df.pivot('ROW','COL','dv2')
sns.heatmap(gridt,cmap='Greys',xticklabels=False, yticklabels=False,ax=ax3,cbar=False)
ax3.set_aspect('equal')
ax3.set_xlabel("")
ax3.set_ylabel("")
ax3.set_title(" Trend Variable")
plt.tight_layout()
plt.savefig("GaussianFFTWithIV.png",dpi=300)
plt.savefig("GaussianFFTWithIV.pdf",dpi=300)

Randomly create missing values

In [ ]:
#Create missing values list
bins = []
for k in range(len(rows['xcoord'])):
    if np.random.random() > .02:
        bins.append(1)
    else:
        bins.append(0)
df['missing'] = bins
print len(bins)
print df['missing'].sum()
print float(df['missing'].sum())/len(bins)

imput = []
for index,row in df.iterrows():
    if row['missing'] ==1:
        imput.append(df['dv'].median())
    else:
        imput.append(row['dv'])
df['dv_m'] = imput
df.head()

### Functions for sampling data, creating a grid of points to account for spatial dependence, calculting inverse distance weighted surface, and developing Random Forest model and comparison metrics

In [ ]:
def performance_metric_r2(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

def performance_metric_mae(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = -1*mean_absolute_error(y_true, y_predict)
    
    # Return the score
    return score

def performance_metric_mse(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = -1*mean_squared_error(y_true, y_predict)
    
    # Return the score
    return score

#score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
#print("Model has a coefficient of determination, R^2, of {:.3f}.".format(score))

def fit_model(X, y,params=None,r2=True,mae=False,mse=False):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(X.shape[0], test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor = GradientBoostingRegressor()#DecisionTreeRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    if params == None:
        params = {'n_estimators': [300,400],
                  'max_depth': list(range(4,35)),
                  'min_samples_leaf':list(range(1,3)),
                  'max_features':["auto"],
                  'loss':['lad'],
                 "learning_rate":[.01,.1],
                 "subsample":[0.5,.75,1.0]}
        #
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    if r2:
        scoring_fnc = make_scorer(performance_metric_r2)
    if mae:
        scoring_fnc = make_scorer(performance_metric_mae)
    if mse:
        scoring_fnc = make_scorer(performance_metric_mse)
    else:
        scoring_fnc = make_scorer(performance_metric_r2)
    # TODO: Create the grid search object
    grid = GridSearchCV(regressor, params, scoring = scoring_fnc,cv=5)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

def createXYGrid(llx,lly,urx,ury,columnCount,rowCount):
    x = np.linspace(llx,urx, columnCount)
    y = np.linspace(lly, ury, rowCount)
    xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
    outCoords = []
    for i in range(len(x)):
        for j in range(len(y)):
            outCoords.append([xv[i,j], yv[i,j]])
    outCoords = np.array(outCoords)
    #plt.scatter(outCoords[:,0],outCoords[:,1])
    #plt.grid(True)
    #plt.axes().set_aspect('equal', 'datalim')
    
    return outCoords

def getDatasetCoords(df,yindex,xindices,coordindices,gcoords,distance= 'minkowski',p=-.5,inv=False,invp=1.0):
    if len(xindices) >0:
        X = df.iloc[:,xindices].values
        print X.shape
    coords =  df.iloc[:,coordindices].values
    Y = df.iloc[:,yindex].values
    if distance == 'minkowski':
        sp_dist = spatial.distance.cdist(coords,gcoords, distance,p=p)
    else:
        sp_dist = spatial.distance.cdist(coords,gcoords, distance)#,p=)
    sp_dist = np.square(sp_dist)
    if inv == True:
        sp_dist = np.divide(1.0,np.power(sp_dist,invp))
    #intern_dist = spatial.distance.cdist(coords,coords, 'euclidean')
    if len(xindices) >0:
        X=np.concatenate((X, sp_dist), axis=1)
    else:
        X = sp_dist
    return X, Y

def getDatasetList(df,coordindices,gcoords):
    coords =  df.iloc[:,coordindices].values
    np.append(coords,gcoords)
    return coords


def getDatasetSample(df,nsample,yindex,xindices,coordindices,gcoords):
    dataset_sub = df.sample(n=nsample, replace=False).copy()
    X,Y = getDatasetCoords(dataset_sub,yindex,xindices,coordindices,gcoords)
    return dataset_sub, X, Y

def getDatasetSampleSub(df,nsample,yindex,xindices,coordindices,subsamp):
    dataset_sub = df.sample(n=nsample, replace=False).copy()
    dataset_sub_sub = dataset_sub.sample(n=subsamp,replace=False).copy()
    gcoords =  dataset_sub_sub.iloc[:,coordindices].values
    X,Y = getDatasetCoords(dataset_sub,yindex,xindices,coordindices,gcoords)
    return dataset_sub,gcoords, X, Y


def getDatasetPred(df,yindex,xindices):
    X = df.iloc[:,xindices].values
    #print X.shape
    Y = df.iloc[:,yindex].values
    return X, Y


def getDatasetSampleNoGrid(df,nsample,yindex,xindices):
    dataset_sub = df.sample(n=nsample, replace=False).copy()
    X,Y = getDatasetPred(dataset_sub,yindex,xindices)
    return dataset_sub,X,Y

def inverseDistanceValues(points,datalist,gridpoints,power=2,nn=12):
    tree = spatial.KDTree(points)
    q_res = tree.query(gridpoints,nn)
    outVals = []
    for i in range(0,len(q_res[0])):
        if len(q_res[0][i][np.where(q_res[0][i]==0.0)]) >0:
            res = datalist[q_res[1][i]][0]
            outVals.append(res)
        else:
            invdist = np.divide(1.0,np.power(q_res[0][i],2))
            value = datalist[q_res[1][i]]
            res = np.divide(np.sum(np.multiply(value,invdist)),np.sum(invdist))
            outVals.append(res)
        if np.isnan(res):
            print invdist
            print value
            print np.sum(invdist)
            print np.multiply(value,invdist)
            print np.sum(np.multiply(value,invdist))
            print i
    return np.array(outVals)

In [ ]:
print df.head()
yind = [6]
xyind = [4,5]
dv = [8,9]
dvxy = [8,9,4,5]
#dv = [2]
#dvxy = [2,4,5]

In [ ]:

llx = min(df['xcoord'])

lly = min(df['ycoord'])

urx = max(df['xcoord'])

ury = max(df['ycoord'])

gridCoords = createXYGrid(llx,lly,urx,ury,4,4)
#df,nsample,yindex,xindices,coordindices,gcoords
#dataset_sub,X,Y= getDatasetSample(df,100,yind,dvxy,xyind,gridCoords)
#df,nsample,yindex,xindices):
dataset_sub,X,Y= getDatasetSampleNoGrid(df,100,yind,dvxy)
plt.scatter(dataset_sub['xcoord'],dataset_sub['ycoord'])
#plt.scatter(gridCoords[:,0],gridCoords[:,1],c='r')
plt.grid(True)
plt.axes().set_aspect('equal')

#df,yindex,xindices,coordindices,gcoords,distance= 'minkowski',p=-.5,inv=False,invp=1.0
#d='euclidean'
#X,Y = getDatasetCoords(dataset_sub,yind,dvxy,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)
print X.shape
sns.set()
#for k in range(len(X)):
    #if np.random.random() > .5:
        #X[k][0]=-100000

#x_pred,y_pred = getDatasetCoords(df,yind,dvxy,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)
x_pred,y_pred = getDatasetPred(df,yind,dvxy)
#for k in range(len(x_pred)):
    #if np.random.random() > .5:
        #x_pred[k][0]=-100000
        

mod = GradientBoostingRegressor(n_estimators=400, learning_rate=0.01, max_features='auto',max_depth=10,min_samples_leaf=2,
                                loss='lad',subsample=.7)
mod.fit(X,Y)

df['predicted']=mod.predict(x_pred)
#alldata['msenb']=np.square(alldata['predicted']-alldata['rain'])
tempPred = mod.predict(x_pred)
print performance_metric_mse(y_pred,tempPred)*-1
print mean_squared_error(y_pred, tempPred)
print performance_metric_mae(y_pred,tempPred)*-1
print performance_metric_r2(y_pred,tempPred)

gridRain= df.pivot('ROW','COL','predicted')
#print grid

ax = sns.heatmap(gridRain,cmap='Blues',xticklabels=False, yticklabels=False)
ax.set_aspect('equal')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("Predicted DV")

In [ ]:
def fit_model(X, y,params=None,r2=True,mae=False,mse=False):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(X.shape[0], test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor = GradientBoostingRegressor()#DecisionTreeRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    if params == None:
        params = {'n_estimators': [400],
                  'max_depth': list(range(8,25)),
                  'min_samples_leaf':list(range(1,3)),
                  'max_features':["auto"],
                  'loss':['lad'],
                 "learning_rate":[.01,.1],
                 "subsample":[0.5,.75,1.0]}
        #
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    if r2:
        scoring_fnc = make_scorer(performance_metric_r2)
    if mae:
        scoring_fnc = make_scorer(performance_metric_mae)
    if mse:
        scoring_fnc = make_scorer(performance_metric_mse)
    else:
        scoring_fnc = make_scorer(performance_metric_r2)
    # TODO: Create the grid search object
    grid = GridSearchCV(regressor, params, scoring = scoring_fnc,cv=5)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_


results3 = {"rfmseg":[],"rfmaeg":[],"rfrsqg":[],"rfmsedv":[],"rfmaedv":[],"rfrsqdv":[],"idwmse":[],"idwmae":[],"idwrsq":[],
           "samplesize":[],"gridsize":[]}
samplesizes = range(20,310,10)
ss=100

for i in range(0,100):
    results3["samplesize"].append(ss)
    gs = int(ss*.05)
    results3["gridsize"].append(gs)
    gridCoords = createXYGrid(llx,lly,urx,ury,4,4)
    dataset_sub,X,Y= getDatasetSampleNoGrid(df,100,yindiv,xyind)
    d='euclidean'
    X,Y = getDatasetCoords(dataset_sub,yindiv,xyind,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)
    x_pred,y_pred = getDatasetCoords(df,yindiv,xyind,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)
    
    mod = fit_model(X, Y,r2=False,mae=False,mse=True)
    tempPred = mod.predict(x_pred)
    results3["rfmseg"].append(performance_metric_mse(y_pred,tempPred)*-1)
    results3["rfmaeg"].append(performance_metric_mae(y_pred,tempPred)*-1)
    results3["rfrsqg"].append(performance_metric_r2(y_pred,tempPred))
    

    d='euclidean'
    X,Y = getDatasetCoords(dataset_sub,yind,dvxym,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)

    x_pred,y_pred = getDatasetCoords(df,yind,dvxym,xyind,gridCoords,distance=d,p=3,inv=False,invp=2)
    mod = fit_model(X, Y,r2=False,mae=False,mse=True)
    tempPred = mod.predict(x_pred)
    results3["rfmsedv"].append(performance_metric_mse(y_pred,tempPred)*-1)
    results3["rfmaedv"].append(performance_metric_mae(y_pred,tempPred)*-1)
    results3["rfrsqdv"].append(performance_metric_r2(y_pred,tempPred))
    
    subset_coords = dataset_sub.iloc[:,xyind].values
    rain_values = np.hstack(dataset_sub.iloc[:,yindiv].values)
    full_grid = df.iloc[:,xyind].values
    tempPred = inverseDistanceValues(subset_coords,rain_values,full_grid)
    results3["idwmse"].append(performance_metric_mse(y_pred,tempPred)*-1)
    results3["idwmae"].append(performance_metric_mae(y_pred,tempPred)*-1)
    results3["idwrsq"].append(performance_metric_r2(y_pred,tempPred))